# <center>Globex Case</center>
<center><font size="3">This is the analysis of the second case from the Data Science Challenges<br>
    Done by: <b>Walid Ismail (wismail@eg.ibm.com)</b></font></center><br>

# 0. Introduction
Globex is a large industrial company with presence in six continents. They produce large industrial products such as ships, power plants, engines, etc. The CFO is examining profitability for numerous projects around the world, starting with projects beginning in 2010. They believe that using data and analytics can help identify projects that may be risky and result in a loss of profit. Globex has compiled data on all completed projects with a start date of 2010 or later. Globex needs the following:

1. Determine the key predictors of a project’s “PROFIT_PLAN”.
2. Develop an actionable, innovative strategy for the client.
3. Prepare a deliverable that is clear and concise, outlining each of your key recommendations.


# 1. Get the Watson Studio project token to use it to access the dataset

In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='01cb8ab3-b48d-441d-8d33-edc6caae425d', project_access_token='p-d36f9300425b2c63be8cd6c20d68a07ef91ec977')
pc = project.project_context

In [ ]:
# Import main needed libraries
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import matplotlib.pyplot as plt
%matplotlib inline
#%config InlineBackend.print_figure_kwargs={'bbox_inches':None}

import seaborn as sns
sns.set_style("white")
sns.set_style("ticks")

# 2. Load the data file

In [ ]:
pd.set_option('display.width', 3000)
pd.set_option('display.max.columns', 500) # display up to 500 colums without "..." use horizontal scrolling instead
pd.set_option('display.max.rows', 50)

# Fetch the Customer file
my_file = project.get_file("Globex_Data_Set.csv")

# Read the EXCEl data file from the object storage into a pandas DataFrame
my_file.seek(0)
globex_df = pd.read_csv(my_file)
print("\nNumber of projects: {}".format(len(globex_df)))
globex_df.head()

# 3. Exploratory Data Analysis
Lets now look at some exploratory information about our dataset and the variables in it to get a flovor of its different aspects. We will look at:<br>
- Number of observations<br>
- Number of columns<br>
- Types of columns<br>
- Does our dataset has empty cells or not?
- Descriptive statistics for each variable

In [ ]:
globex_df.shape # dimensions of data set

In [ ]:
globex_df.isnull().sum() # check if any cells have missing values

In [ ]:
globex_df.info() # check data type of each field

In [ ]:
globex_df.describe() # view statisical measures of variables

In [ ]:
globex_df.plot(kind='scatter', x='CTRCT_AMT_PLAN', y=['PROFIT_PLAN'])

# 4. Feature Engineering / Data Preparation
From our data inspection we can observe the following about the variables / features / target:
<ul>
    <li>We have 27 attribuites in our dataset (26 possible features and one target)</li>
    <li>Since Globex is seeking information on key predictors for 'PROFIT_PLAN', then our target variable for this analysis will be the variable <b>PROFIT_PLAN</b></li>
    <li>All attributes are in numeric format.</li>
    <li>We have <b>15</b> numeric variables that are continious in nature <b>(PROJ_DURN, CNTRY_DFT, FTE_INT, FTE_CONTR, CC, INSUR_BEFORE_AMT, INSUR_AFTER_AMT, INSUR_AMT, TAX_RT, CAP_UTIL, GNT_INVEST, LIQ_RSK, DIM_CONSTR, IPI, CTRCT_AMT_PLAN)</b>. These variables will require scaling before they are used in modeling</li>
    <li><b>11</b> numeric variables <b>PROJ_CD_TFD, R1, R2, R3, R4, R5, R6, R7, CNTRY, REGION, CUST</b> that are numeric encoding of categorical variables so they  need to be tranformed to nominal variables of values (0,1)</li>
</ul>

# 5. Feature Selection
We are going to use several methods to try and discover the features that will be strong predictors for customer churn. Specifically we will try the following techniques:

- <b>Correlations</b>: this will help identify features pairs that are strongly correlated with each other so that we can remove one feature of them.
- Also correlation will identify numeric features that are highly correlated with the target variable to select as predictors for modeling
- <b>Box Plots</b> help us visualize the relationship between categorical variables and PROFIT_PLAN to see if there is variation in PROFIT_PLAN as a result of change in values for each of the categorical features. This should help us identify which categorical features qualify as a predictors for PROFIT_PLAN.
- <b>SelectKBest method</b>: this is a scikit learn object that ranks a list of features according to their predictice impact by assigning a score to each feature

## 5.1 Correlations
Now we check the Pearsom correlation between the continious features to see if any of the features pairs are highly correlated and as a result we can exclude one of the features of the pairs

In [ ]:
#check for correlations between numeric features. Rule, if two features that are highly correlated one should be dropped
# Does scaling make a difference in result ???? No, no need to scale features before running corr(), you will get the same scores. Try it yourself!

numeric_features = ['PROJ_DURN', 'CNTRY_DFT', 'FTE_INT', 'FTE_CONTR', 'CC', 
                    'INSUR_BEFORE_AMT', 'INSUR_AFTER_AMT', 'INSUR_AMT', 
                    'TAX_RT', 'CAP_UTIL', 'GNT_INVEST', 'LIQ_RSK', 'DIM_CONSTR', 'IPI', 'CTRCT_AMT_PLAN', 'PROFIT_PLAN']

#plt.style.use('default')
plt.figure(figsize=(12, 10))
sns.heatmap(globex_df[numeric_features].corr(),  cmap="Blues", annot=True, fmt=".2f")

The heat map above shows a maximum correlation of 0.67 between <b>employ</b> and <b>age</b>. All other features pairs have lower correlation. This correlation is not strong enough to warrant exclusion of any of the numeric features based on correlation alone.

## 5.2 Box plots to compare categorical variables against the target variables
Let us check the 11 categorical features that we have in the dataset against <b>PROFIT_PLAN</b>. The categorical features are as follows:
    <b>PROJ_CD_TFD, R1, R2, R3, R4, R5, R6, R7, CNTRY, REGION, CUST</b>

In [ ]:
catgorical_features = ['PROJ_CD_TFD', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'CNTRY', 'REGION', 'CUST']
target_feature = 'PROFIT_PLAN'

font = {'weight' : 'normal',
        'size'   : 16}

plt.rc('font', **font)
plt.rc('axes', titlesize=30)     # fontsize of the axes title

fig, axs = plt.subplots(4, 3, sharex=False, sharey=True, figsize=(30,30))
plt.subplots_adjust(hspace=0.4)

for i, feat in enumerate(catgorical_features):
    ax = globex_df.boxplot(target_feature, by=catgorical_features[i], ax = axs[int(i/3), int(i%3)], showmeans=True)
    ax.set_title('')
    fig2 = ax.get_figure()
    fig2.suptitle('')
     
fig.suptitle('PROFIT_PLAN grouped by different categroical variables', fontsize=24, y=0.95)

for i in range(11, 12):
    fig.delaxes(axs[int(i/3), int(i%3)])
   
plt.show()

The cross tabulation above provides the following information about the relationship between each of the nominal features and churn:
<ul>
    <li>Each of the features <b>marital, gender, tollfree, multline, callid, confer</b> have the same distribution whether they are 0 or 1 on customer churn so we can conclude that they will be <b>weak</b> predictors for customer churn &#x274C;</li>
    <li>Each of the features <b>equip, internet, ebill</b> have same distribution when they are 0 vs 1 on customer churn so we should take only one of them and use as a predictor. We shall select <b>equip</b></li>
    <li>Each of the features <b>wireless, voice, pager</b> have same distribution when they are 0 vs 1 on customer churn so we should take only one of them and use as a predictor. We shall select <b>wireless</b></li>
    <li><b>Callcard</b> havs an opposite distribution to all other nominal features where churn decreases as customer buy callcards. Accordingly, we shall select <b>callcard</b> as a predictor</li>       
</ul>

## 5.5 SelectKBest Method
SelectKBest is an object from Scikit Learn library that we can used for automatic feature selection. SelectKBest ranks all features based on their importance as predictors for the target variable using different statistical methods. We use it here as a validation of our selected features. As we can see from the result below, our selected features match the top 6 features ranked by SelectKBest.

In [ ]:
# Feature Extraction with Univariate Statistical Tests
pd.set_option('display.expand_frame_repr', False)
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif

all_features = ['PROJ_CD_TFD', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 
            'PROJ_DURN', 'CNTRY_DFT', 'CNTRY', 'REGION', 'FTE_INT', 'FTE_CONTR', 'CC', 'CUST', 
            'INSUR_BEFORE_AMT', 'INSUR_AFTER_AMT', 'INSUR_AMT', 'TAX_RT', 'CAP_UTIL', 
            'GNT_INVEST', 'LIQ_RSK', 'DIM_CONSTR', 'IPI', 'CTRCT_AMT_PLAN']

catgorical_features = ['PROJ_CD_TFD', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 
                        'CNTRY', 'REGION', 'CUST']

numeric_features = ['PROJ_DURN', 'CNTRY_DFT', 'FTE_INT', 'FTE_CONTR', 'CC', 
                    'INSUR_BEFORE_AMT', 'INSUR_AFTER_AMT', 'INSUR_AMT', 
                    'TAX_RT', 'CAP_UTIL', 'GNT_INVEST', 'LIQ_RSK', 'DIM_CONSTR', 'IPI', 'CTRCT_AMT_PLAN']

main_features = ['PROJ_CD_TFD', 'R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 
            'PROJ_DURN', 'CNTRY_DFT', 'CNTRY', 'REGION', 'FTE_INT', 'FTE_CONTR', 'CC', 'CUST', 
            'TAX_RT', 'CAP_UTIL', 
            'GNT_INVEST', 'LIQ_RSK', 'DIM_CONSTR', 'IPI', 'CTRCT_AMT_PLAN']

target = 'PROFIT_PLAN'

# Does scaling make a difference in result ???? No, no need to scale features before running SelectKBest(), you will get the same scores. Try it yourself!

features_to_test = numeric_features

X = globex_df[features_to_test]
Y = globex_df[target]

# feature extraction
selector = SelectKBest(score_func=f_regression, k=2)
selector.fit(X, Y)

#scores = -np.log10(selector.pvalues_)
scores = selector.scores_

# summarize scores
np.set_printoptions(precision=3, suppress=True)
pd.options.display.float_format = '{:.3f}'.format
scored_features = pd.DataFrame({'Feature': features_to_test, 'Score' : scores})
scored_features.sort_values(by = 'Score', ascending=False, inplace=True)
scored_features = scored_features.reset_index(drop=True)
scored_features.head(10)

#features = fit.transform(X) # get the array of observations for top 5 features, NICE!!
#print(features)

## 5.6 ANOVA Test

In [ ]:
import statsmodels.api as sm
from scipy import stats

X = globex_df['R7']
y = globex_df[target]

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

## 5.7 Chi-Squared Test

In [ ]:
import scipy.stats as stats
tab = pd.crosstab(globex_df['R4'], globex_df['R5'])
print(tab)
print(stats.chi2_contingency(tab))

## 5.6 Selected Features
Based on the analysis above we have identified the following features as predictors for <b>PROFIT_PLAN</b> to be used in our modeling:<br>
- 1
- 2
- 3
- 4
- 5
- 6
- 7
- 8
- 9

# 6. Modeling
In the section we will focus finding the best model in order to predict customer churn for WW Communications. Since <b>churn</b> is a nominal variable (0,1) then our problem is mainly a classification problem. We will try different classification models to see which one yields the highest training accuracy, then we will fine tune our most accurate model in order to arrive at the bet testing accuracy.

## 6.0 Transform Categorical Variables to Nominal Features

In [ ]:
selected_features = ['PROJ_CD_TFD', 'R1', 'R2', 'R3', 'R5', 'R6', 'R7', 'CNTRY',
                    'GNT_INVEST', 'CTRCT_AMT_PLAN'] + [target]

globex_df2 = globex_df[selected_features]
globex_df2 = pd.get_dummies(globex_df2, columns = ['PROJ_CD_TFD', 'R1', 'R2', 'R3', 'R5', 'R6', 'R7', 'CNTRY'])
globex_df2

## 6.1 Feature Scaling
Before modeling we need to do feature scaling in order to standardize our measurement scale for all numeric features

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler

y = globex_df2[target]
X = globex_df2.drop(target, axis=1)

# ALWAYS SPLIT FIRST BEFORE SCALING TO AVOID PREDICTOR LEAKAGE !!!!!

#Train Test Split, reserve test subset for finally accuracy and other metrics calculations
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(X_train) # fit the scaler on training data then apply to training and test data
X_train = scaler.transform(X_train)
X_test =  scaler.transform(X_test)
print('Done')

## 6.2 Modeling (using different models)
Now lets try three different classification models and see which one yields the best training accuracy. We will try the following three models:
- Logistic Regression
- Support Vector Classifier
- Random Forest Classifier

In [ ]:
from math import sqrt
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor

# using specific random state to make sure we always get same accuracy result if we execute this cell multiple times or on multiple machines

lm = LinearRegression()
scores = cross_val_score(lm, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print("Linear regression RMSE is: {:,}".format(sqrt(-scores.mean())))

svr = svm.SVR()
scores = cross_val_score(svr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print("SVM RMSE is: {:,}".format(sqrt(-scores.mean())))

dcr = DecisionTreeRegressor()
scores = cross_val_score(dcr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print("Descision Tree RMSE is: {:,}".format(sqrt(-scores.mean())))

knnr = KNeighborsRegressor(n_neighbors=30)
scores = cross_val_score(dcr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print("KNeighborsRegressor RMSE is: {:,}".format(sqrt(-scores.mean())))

poly = PolynomialFeatures(2)
X_transform = poly.fit_transform(X_train)
scores = cross_val_score(lm, X_transform, y_train, cv=5, scoring='neg_mean_squared_error')
print("Ploynomial RMSE is: {:,}".format(sqrt(-scores.mean())))

## 6.3 Fine tuning the selected model hyperparameters
Since Logistic Regression yielded the best training accuracy then we will select that model as our estimator of churn and now we need to identify the best hyperparameters to initialize the model in order to yield the highest possible testing accuracy

In [ ]:
from sklearn.model_selection import GridSearchCV
#from sklearn.model_selection import RandomizedSearchCV
import numpy as np

C_options = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
solver_options = ['newton-cg', 'lbfgs', 'liblinear', 'sag']
class_weight_options = [None, 'balanced']
param_grid = dict(C = C_options, solver = solver_options, class_weight = class_weight_options)

clf = GridSearchCV(LogisticRegression(), param_grid)

clf.fit(X_train, y_train)

best_model = clf.best_estimator_
best_model

# 7. Model Evaluation
In this section we check the tuned model testing accuracy. We also take a look at the relative importance of the model features in predicting customer churn. We conclude with an overview of the confusion matrix

## 7.1 Testing accuracy

In [ ]:
y_pred = best_model.predict(X_test)

from sklearn.metrics import classification_report

print('\nAccuracy of tuned model on test set: {0:.2f}\n'.format(best_model.score(X_test, y_test)))

print(classification_report(y_test, y_pred))

## 7.2 Displaying the modeling predictors sorted by coefficent
In this section we take a look at the influence of the model features on customer churn.<br>
- Features with +ve coefficient drive customer churn (i.e. churn = 1)
- Features with -ve coefficient drive customer retention (i.e churn = 0)

In [ ]:
import pandas as pd

# model is the trained model
importances = best_model.coef_[0]
print(importances)

# Extract the feature importances into a dataframe
feature_results = pd.DataFrame({'feature': features, 
                                'importance': importances})

# Show the top 10 most important
feature_results = feature_results.sort_values('importance', ascending = False).reset_index(drop=True)

feature_results.head(10)

In [ ]:
#plt.style.use('fivethirtyeight')
#plt.style.use('default')
# Plot the most important features in a horizontal bar chart
feature_results.plot(x = 'feature', y = 'importance', edgecolor = 'k', kind='barh', color = 'blue', figsize=(6, 3))
plt.xlabel('Relative Importance', size = 12)
plt.ylabel('')
plt.title('Feature Importances', size = 12)
plt.grid(False)
plt.show()

## 7.3 Confsuion Matrix
Finally we take a look the resulting confusion matrix and evaluate our model performance in predicting customer churn vs retention. Specifically we will exanme the following metrics:

Precision (the positive predictive value) = TP / (TP+FP)
<br>
Specificity (the true negative rate) = TN / (FP+TN)
<br>
Accuracy (the fraction of the total sample that is correctly identified) = (TP+TN) / (TP+TN+FP+FN)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.grid(False)


    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred, [1, 0])
np.set_printoptions(precision=2)

tp, fn, fp, tn = cnf_matrix.ravel()

accuracy = (tn+tp) / (tn + fp + fn + tp)
precision = tp / (tp+fp)
specifity = tn / (fp+tn)

print("Accuracy = {0:.2f}".format(accuracy))
print("Precision = {0:.2f}".format(precision))
print("Specifity = {0:.2f}".format(specifity))

# Plot non-normalized confusion matrix
plt.figure(figsize=(4,4))
plot_confusion_matrix(cnf_matrix, classes=['churn=1','churn=0'],  title='Confusion matrix')

# 8. Conclusions and Recommendations
Based on our analysis we provide the following conclusions and recommendations for the future:
- The strongest three predictors affecting consumer retention (churn = 0) are as follows:
    - tenure
    - address
    - employ
- The strongest three predictors affecting consumer churn are as follows:
    - income
    - equip
    - age
- Our model is better at predicting retention (churn = 0) than churn (churn = 1). This is due to the dataset containing more observations with churn=0 than churn=1. Our model should become better at predicting churn if the dataset is updated with more observations for +ve churn customers
- Customers in certain address code (range 0-11) have a high churn percentage. This could be due to network issues in theses areas. Futher investigation is advised
- Customers in low income range (0-47) are more likely to leave WW. Further analysis is advised on price plans for these customers
- Customers who have equip = 1 are more likely to leave WW. Futher analysis advised to uncover reasons for this
- Customers with high years of education and employment are more loyal. Should proivde more incentives to attract these types of customers